## html parsing, completed


just collect everything raw from Erowid and assemble it in a list of dicionaries

In [1]:
!pip install rdkit-pypi

     |████████████████████████████████| 20.6 MB 1.5 MB/s 


In [83]:
# for files
import os
import pickle
from google.colab import drive, files

# to get source
import requests

# rabbit hunting with a howitzer
from bs4 import BeautifulSoup

# I don't know, maybe needed for some reason
import numpy as np

# for parsing
import re

# for MolToSmiles
import rdkit
from rdkit import Chem

In [4]:
# base links
tihkal = "https://www.erowid.org/library/books_online/tihkal/tihkal.shtml"
pihkal = "https://www.erowid.org/library/books_online/pihkal/pihkal.shtml"
tihkalBase = "https://www.erowid.org/library/books_online/tihkal/"
pihkalBase = "https://www.erowid.org/library/books_online/pihkal/"
erowidBase = "https://www.erowid.org/"

In [35]:
# tihkal pages
tihkalSource = requests.get(tihkal).text
soup = BeautifulSoup(tihkalSource, 'html.parser')
tihkalLinks = [x.find('a')['href'] for x in soup.find_all('table')[7].find_all('tr')[1:-2]]
tihkalPages = [tihkalBase+x for x in tihkalLinks]

In [36]:
# pihkal pages
pihkalSource = requests.get(pihkal).text
pSoup = BeautifulSoup(pihkalSource, 'html.parser')
pihkalLinks = [x.find('a')['href'] for x in pSoup.find_all('table')[6].find_all('tr')[1:-1]]
pihkalPages = [pihkalBase+x for x in pihkalLinks]

In [37]:
# get the molfile
linkMol = lambda x: (erowidBase +
  BeautifulSoup(requests.get(x).text, 'html.parser').find('a', href = re.compile(r'.*\.mol'))['href'][1:])

# get the text
linkText = lambda x: BeautifulSoup(requests.get(x).text, 'html.parser')

# combine the two
linkAll = lambda x: (linkMol(x), linkText(x))

These are lists of hyperlinks to `.mol` files.

In [39]:
tihkalMolsLinks = list(map(linkAll, tihkalPages))
pihkalMolsLinks = list(map(linkAll, pihkalPages))

In [42]:
tihkalMolsLinks[0][0]

'https://www.erowid.org/chemicals/other/images/archive/al-lad__.mol'

In [44]:
r = requests.get(tihkalMolsLinks[0][0])

Next, a function to get the SMILES string.

In [112]:
# to get smiles from requests response
def getSMILES(raw):
  # raw is a requests response
  try:
    os.remove('myMol.mol')
    with open('myMol.mol', 'w') as molF:
      molF.writelines(raw.content.decode('utf-8'))
    myMol = Chem.MolToSmiles(Chem.MolFromMolFile('myMol.mol'))
  except:
    with open('myMol.mol', 'w') as molF:
      molF.writelines(raw.content.decode('utf-8'))
    myMol = Chem.MolToSmiles(Chem.MolFromMolFile('myMol.mol'))
  return myMol

# to get smiles from requests response text content
def easySMILES(text):
  # text is a string recording the .mol file
  try:
    os.remove('myMol.mol')
    with open('myMol.mol', 'w') as molF:
      molF.writelines(text)
    myMol = Chem.MolToSmiles(Chem.MolFromMolFile('myMol.mol'))
  except:
    with open('myMol.mol', 'w') as molF:
      molF.writelines(text)
    myMol = Chem.MolToSmiles(Chem.MolFromMolFile('myMol.mol'))
  return myMol

In [52]:
getSMILES(r)

'[cH:1]1[cH:2][cH:3][c:14]2[c:13]3[c:4]1[C:6]1=[CH:11][CH:10]([C:19](=[O:20])[N:21]([CH2:22][CH3:25])[CH2:23][CH3:24])[CH2:9][N:8]([CH2:26][CH:27]=[CH2:28])[CH:7]1[CH2:5][c:12]3[cH:16][nH:15]2'

Let's save our progress.

In [66]:
newSoup = BeautifulSoup(tihkalMolsLinks[0][1].text, 'html.parser')

In [68]:
newSoup == tihkalMolsLinks[0][1].text

False

In [72]:
tihkalMolsLinks[0][1].text

'\n\nErowid Online Books : "TIHKAL" - #1 AL-LAD\n\n\n\n\n\n#content-outer-frame { background-color: #ffffff; }\n.content-section a:link { color: #333399; }\n.content-section a:visited { color: #773344; }\n\n\n\n\n\n\n\n\n\xa0\n\n\n\n\n\xa0\n\n\n\xa0\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\xa0\n\n\n\n\n\n\n\nLIBRARY \xa0 books_online \xa0 tihkal\n\n\n\n\n\n\n#1. AL-LAD \n6-NORLYSERGIC ACID, 6-ALLYL-N,N-DIETHYLAMIDE; 6-NORLYSERGAMIDE,\n6-ALLYL-N,N-DIETHYL; N,N-DIETHYLNORLYSERGAMIDE, 6-ALLYL;\nN-(6)-ALLYLNORLYSERGIC ACID, N,N-DIETHYLAMIDE;\n9,10-DIDEHYDRO-6-ALLYL-N,N-DIETHYLERGOLINE-8b-CARBOXAMIDE\n\n\n\n\n\n[3D .mol structure]\n\n\n\nSYNTHESIS: To a solution of 66 mg nor-LSD (see under ETH-LAD for its preparation) in 2 mL freshly\ndistilled DMF under a nitrogen atmosphere, there was added 48 mg\nanhydrous K2CO3 and 30 mg allyl bromide. When TLC analysis indicated\nthat the nor-LSD had been consumed (30 min) all volatiles were removed\nunder a hard vacuum. The resi

In [73]:
tihkalMolsLinks[0][0]

'https://www.erowid.org/chemicals/other/images/archive/al-lad__.mol'

In [74]:
# to clean up and get a string
clean = lambda x: (x[0], x[1].text)

# to save tihkal
with open('tihkalPickle', 'wb') as tihk:
  pickle.dump(list(map(clean, tihkalMolsLinks)), tihk)

# to save pihkal
with open('pihkalPickle', 'wb') as pihk:
  pickle.dump(list(map(clean, pihkalMolsLinks)), pihk)

Let's see how that worked.

In [75]:
with open('tihkalPickle', 'rb') as tihk:
  tihkalFromPickle = pickle.load(tihk)

In [77]:
tihkalFromPickle[0][0]

'https://www.erowid.org/chemicals/other/images/archive/al-lad__.mol'

And on the other one?

In [78]:
with open('pihkalPickle', 'rb') as pihk:
  pihkalFromPickle = pickle.load(pihk)

Looking good! Now let's bite the bullet and finally figure out how to store things in drive with colab.

In [85]:
#trying to mount it
drive.mount('/content/drive')

Mounted at /content/drive


OK, that worked. Now let's see if we can write our files that we already wrote.

In [86]:
!cp pihkalPickle /content/drive/MyDrive/Colab\ Notebooks

Neat. That worked.

In [87]:
!cp tihkalPickle /content/drive/MyDrive/Colab\ Notebooks

OK, last steps: let's finalize those changes, and then remount the Drive and then see if we can load those objects directly from drive.

In [88]:
drive.flush_and_unmount()

Mounting again.

In [89]:
drive.mount('/content/drive')

Mounted at /content/drive


Sweet. Let's see if I can load those silly fucking objects directly from Drive.

In [91]:
with open('/content/drive/MyDrive/Colab Notebooks/tihkalPickle', 'rb') as that:
  tihkalFromDrive = pickle.load(that)

In [93]:
tihkalFromDrive == tihkalFromPickle

True

Yes! But let's be sure.

In [94]:
with open('/content/drive/MyDrive/Colab Notebooks/pihkalPickle', 'rb') as this:
  pihkalFromDrive = pickle.load(this)

In [95]:
pihkalFromDrive == pihkalFromPickle

True

Now we are cooking with gas! Ope, almost forgot to clean up after myself:

In [96]:
drive.flush_and_unmount()

But wait! Our celebration was premature. Our most important data is still on the Internet. We need to bring it in and write it.

In [100]:
type(r.content.decode('utf-8'))

str

Here is a function to get as a string from a `.mol` link like the ones we produced.

In [101]:
getMolText = lambda x: requests.get(x).content.decode('utf-8')

In [103]:
dir()

['BeautifulSoup',
 'Chem',
 'In',
 'Out',
 '_',
 '_100',
 '_102',
 '_22',
 '_23',
 '_25',
 '_26',
 '_27',
 '_30',
 '_31',
 '_32',
 '_33',
 '_34',
 '_38',
 '_40',
 '_41',
 '_42',
 '_46',
 '_52',
 '_56',
 '_57',
 '_58',
 '_59',
 '_61',
 '_62',
 '_63',
 '_64',
 '_67',
 '_68',
 '_69',
 '_70',
 '_72',
 '_73',
 '_76',
 '_77',
 '_79',
 '_80',
 '_81',
 '_92',
 '_93',
 '_95',
 '_97',
 '_98',
 '__',
 '___',
 '__builtin__',
 '__builtins__',
 '__doc__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 '_dh',
 '_exit_code',
 '_i',
 '_i1',
 '_i10',
 '_i100',
 '_i101',
 '_i102',
 '_i103',
 '_i11',
 '_i12',
 '_i13',
 '_i14',
 '_i15',
 '_i16',
 '_i17',
 '_i18',
 '_i19',
 '_i2',
 '_i20',
 '_i21',
 '_i22',
 '_i23',
 '_i24',
 '_i25',
 '_i26',
 '_i27',
 '_i28',
 '_i29',
 '_i3',
 '_i30',
 '_i31',
 '_i32',
 '_i33',
 '_i34',
 '_i35',
 '_i36',
 '_i37',
 '_i38',
 '_i39',
 '_i4',
 '_i40',
 '_i41',
 '_i42',
 '_i43',
 '_i44',
 '_i45',
 '_i46',
 '_i47',
 '_i48',
 '_i49',
 '_i5',
 '_i50',
 '_i51',
 '_i52',


In [106]:
rawMolTihkal = [(getMolText(x), y) for x,y in tihkalFromDrive]
rawMolPihkal = [(getMolText(x), y) for x,y in pihkalFromDrive]

In [109]:
print(rawMolTihkal[1][0])

Molecule-1
  WLViewer          3D                             0

 48 49  0  0  0  0  0  0  0  0999 V2000
   13.3729   -6.6098   -0.5136 C   0  0  0  0  0  0  0  0  0  1
   12.0365   -6.6884   -0.5150 C   0  0  0  0  0  0  0  0  0  2
   11.3459   -7.9439   -0.0676 C   0  0  0  0  0  0  0  0  0  3
   12.0581   -9.0008    0.3377 C   0  0  0  0  0  0  0  0  0  4
   14.1005   -7.7760   -0.0664 C   0  0  0  0  0  0  0  0  0  5
   13.4958   -8.8709    0.3231 C   0  0  0  0  0  0  0  0  0  6
   14.4722   -9.8252    0.6970 N   0  0  0  0  0  0  0  0  0  7
   15.7716   -9.2107    0.5033 C   0  0  0  0  0  0  0  0  0  8
   15.5358   -7.9748    0.0423 C   0  0  0  0  0  0  0  0  0  9
   16.6003   -6.9746   -0.3079 C   0  0  0  0  0  0  0  0  0 11
   17.9580   -7.5838    0.0979 C   0  0  0  0  0  0  0  0  0 12
   19.0908   -6.7422   -0.3102 N   0  0  0  0  0  0  0  0  0 13
   20.3206   -7.5083    0.1307 C   0  0  0  0  0  0  0  0  0 14
   21.6732   -7.0170   -0.4424 C   0  0  0  0  0  0  0  0  0 15

In [114]:
easySMILES(rawMolTihkal[0][0])

'[cH:1]1[cH:2][cH:3][c:14]2[c:13]3[c:4]1[C:6]1=[CH:11][CH:10]([C:19](=[O:20])[N:21]([CH2:22][CH3:25])[CH2:23][CH3:24])[CH2:9][N:8]([CH2:26][CH:27]=[CH2:28])[CH:7]1[CH2:5][c:12]3[cH:16][nH:15]2'

Seems to be working. Not sure what those dollar signs at the bottom were about. Let's go ahead and write these.

In [115]:
drive.mount('/content/drive/')

Mounted at /content/drive/


In [116]:
# the tihkal data with the mol strings in it
with open('/content/drive/MyDrive/Colab Notebooks/rawMolTihkalDrive', 'wb') as tihk:
  pickle.dump(rawMolTihkal, tihk)

# the pihkal data with the mol strings in it
with open('/content/drive/MyDrive/Colab Notebooks/rawMolPihkalDrive', 'wb') as pihk:
  pickle.dump(rawMolPihkal, pihk)

OK, this next thing really is the last thing for this *current notebook*: we are going to read in these objects and check that they equal what we started with.

In [118]:
with open('/content/drive/MyDrive/Colab Notebooks/rawMolTihkalDrive', 'rb') as tihk:
  rawMolTihkFromDrive = pickle.load(tihk)

with open('/content/drive/MyDrive/Colab Notebooks/rawMolPihkalDrive', 'rb') as pihk:
  rawMolPihkFromDrive = pickle.load(pihk)

In [120]:
(rawMolPihkFromDrive, rawMolTihkFromDrive) == (rawMolPihkal, rawMolTihkal)

True

Success! This worked. Just a reminder: those two files are `rawMolTihkalDrive` and `rawMolPihkalDrive`.

In [121]:
drive.flush_and_unmount()